# Лабораторная работа 5

## Применение архитектуры трансформеров для анализа тональности текста

Загрузим необходимые библиотеки для pytorch transformers

In [1]:
!pip install transformers -U

In [2]:
!pip install transformers[torch]

In [3]:
!pip install accelerate -U

In [4]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import tqdm
from tqdm.auto import tqdm
import torch.cuda
from IPython.display import clear_output
import matplotlib.pyplot as plt
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer
import torchtext
import pandas as pd

from transformers import AutoModel
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer

from sklearn.metrics import accuracy_score

import numpy as np

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [9]:
df_train = pd.read_csv("train.csv", engine='python')
df_test = pd.read_csv('test.csv', engine='python')

In [10]:
from transformers import AutoTokenizer

Я буду использовать предобученную модель 'Rubert-tiny', которая является моделью архитектуры 'Bert', обученной на русском корпусе текстов. В качестве токенизатора буду использовать предобученный токенизатор rubert-tiny. Я не буду обучать всю модель целиком, я возьму предобученную модель и дообучу выходной линейный слой, который будет выдавать логиты двух классов

In [13]:
tokenizer = AutoTokenizer.from_pretrained('cointegrated/rubert-tiny2')
model = AutoModelForSequenceClassification.from_pretrained('cointegrated/rubert-tiny2')
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(83828, 312, padding_idx=0)
      (position_embeddings): Embedding(2048, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-2): 3 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-12, 

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X = list(df_train['text'])
y = list(df_train['class'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [16]:
X_train_tokenized = tokenizer(X_train, padding = True, truncation = True, max_length = 256)
X_test_tokenized = tokenizer(X_test, padding = True, truncation = True, max_length = 256)

In [17]:
X_train_tokenized[0]

Encoding(num_tokens=135, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [18]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

Создадим класс TweetsDataset для обёртки датасета с твитами

In [19]:
class TweetsDataset(Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    if self.labels:
      item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.encodings['input_ids'])

In [20]:
train_dataset = TweetsDataset(X_train_tokenized, y_train)
test_dataset = TweetsDataset(X_test_tokenized, y_test)

Объект класса TweetsDataset: как мы видим, в нём хранятся индексы слов, индекс типа токенов, маска для attention и метка класса

In [21]:
train_dataset[0]

{'input_ids': tensor([    2, 50776, 10934,     5,     5,     5,   299, 10227, 10030,  5489,
         25628,     5,    12,   897,    30,    19,    19,    87,    18,  1293,
            19,    53,  1012,   572,  1265,   572,   599,   587, 27006,   750,
             3,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [22]:
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy}

In [23]:
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=1,
    per_device_train_batch_size=8

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

In [24]:
trainer.train()

Step,Training Loss
500,0.088900
1000,0.010800
1500,0.003400
2000,0.007500
2500,0.004300
3000,0.002200
3500,0.004100
4000,0.004000
4500,0.003900
5000,0.003800


TrainOutput(global_step=18147, training_loss=0.005736427572317967, metrics={'train_runtime': 524.6205, 'train_samples_per_second': 276.72, 'train_steps_per_second': 34.591, 'total_flos': 282270168279540.0, 'train_loss': 0.005736427572317967, 'epoch': 1.0})

In [25]:
trainer.evaluate()

{'eval_loss': 0.0012890738435089588,
 'eval_accuracy': 0.9998346834187469,
 'eval_runtime': 23.4127,
 'eval_samples_per_second': 1550.188,
 'eval_steps_per_second': 193.784,
 'epoch': 1.0}

Точность составила порядка 0.99, что очень и очень хорошо

'Bert' --- это энкодер от трансформера, с некоторыми фишками, заточенными для решения задачи классификации. Его можно обучить на достаточно большом корпусе текстов, получив большую языковую модель, которую легко потом можно обучить решать задачи классификации текста, question-answering и многих других

Transfer Learning позволяет использовать предобученную большую языковую модель, для решения своих задач, обучив последний слой. Также для 'Bert' есть уменьшенные модели с меньшим числов параметров, а также модификации 'Bert', использующие дистилляцию, что позволяет использовать данную архитектуру на домашнем ПК или даже телефоне. Поэтому изучение данной языковой модели очень важно

In [26]:
df_val = pd.read_csv('test_stud_data.csv')
df_val.head()

,id,text
0,181467,RT @TukvaSociopat: Максимальный репост! ))) #є...
1,181468,чтоб у меня з.п. ежегодно индексировали на инд...
2,181469,@chilyandlime нехуя мне не хорошо !!! :((((
3,181470,"@inafish нее , когда ногами ахахах когда?ахаха..."
4,181471,"Хочу сделать как лучше, а получаю как всегда. :("


Теперь сделаем предикт на тестовой выборке и загрузим её в файл

In [27]:
def get_sentiment(text):
  with torch.no_grad():
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(model.device)
    proba = torch.sigmoid(model(**inputs).logits).cpu().numpy()[0]
  return model.config.id2label[proba.argmax()]

In [28]:
df_val['prediction'] = df_val['text'].apply(get_sentiment)

In [29]:
df_val.head()

,id,text,prediction
0,181467,RT @TukvaSociopat: Максимальный репост! ))) #є...,LABEL_1
1,181468,чтоб у меня з.п. ежегодно индексировали на инд...,LABEL_0
2,181469,@chilyandlime нехуя мне не хорошо !!! :((((,LABEL_0
3,181470,"@inafish нее , когда ногами ахахах когда?ахаха...",LABEL_0
4,181471,"Хочу сделать как лучше, а получаю как всегда. :(",LABEL_0


In [32]:
df_val['prediction'] = df_val['prediction'].replace({'LABEL_1': 1, 'LABEL_0': 0})

In [33]:
df_val.head()

,id,text,prediction
0,181467,RT @TukvaSociopat: Максимальный репост! ))) #є...,1
1,181468,чтоб у меня з.п. ежегодно индексировали на инд...,0
2,181469,@chilyandlime нехуя мне не хорошо !!! :((((,0
3,181470,"@inafish нее , когда ногами ахахах когда?ахаха...",0
4,181471,"Хочу сделать как лучше, а получаю как всегда. :(",0


In [36]:
df_val['prediction'].value_counts()

prediction
1    11454
0    11229
Name: count, dtype: int64

In [37]:
df_val.to_csv('test_4.csv')